# ПОСТАНОВКА ЗАДАЧИ
Необходимо создать два основных Класса:
* Hub (синглтон, класс объекта нашего склада). Содержит атрибуты: hub (хранит экземпляр единственного объекта), _date (дата создания склада), _items (список предметов в виде контейнера).
* Item (класс предметов, хранящихся на складе). Содержит атрибуты: _id (формируется автоматически при добавлении предмета на склад), name (название предмета), description (описание предмета), date_add (дата добавления предмета), _tags (список характеристик предмета в виде списка произвольного размера), _cost (стоимость предмета). Список характеристик может быть, например: ["хрупкий", "стекло", "Россия"].

Hub должен быть синглтоном (при любом вызове Hub() возвращается один и тот же инстантс объекта) и содержать следующие методы:
1) добавление предмета в список _items,
2) удаление предмета из списка по id,
3) обращение к предмету по индексу (изменить "магический метод" __getitem__),
4) подсчет количества предметов на складе (изменить "магический методв" __len__),
5) вывод информации о складе (переопределить "магический метод" __str__), в котором указать дату создания склада, количество предметов на складе,
6) метод find_by_tags([tags]), который возвращает контейнер, который содержит все предметы из items у который есть ВСЕ теги из _tags (для этого будет полезно использовать метод в is_tagged(tag) из Items)
7) методы set_date() и get_date() для установки даты на складе
8) метод find_by_date(), который возвращает лист всех Item, подходящих по дате. Реализуйте его таким образом, что если в метод передаётся только одна дата, то возвращаются все items с датой раньше или равной ей, а если передаётся две даты, тогда все items с датой в этом промежутке (для этого можно использовать *args и смотреть сколько аргументов в явном виде передали). В случае если передали слишком много параметров - бросайте ошибку.
9) Метод find_most_valuable(amount=1) который вернёт первые amount самых дорогих предметов на складе. Если предметов на складе меньше чем amount - верните их все

Класс Item должен иметь уникальный _id и поддерживать следующие методы:
1) вывод инфорации об отдельном предмете (переопределить "магический метод" __str__),
2) добавление одного или нескольких тэгов add_tags(tags) к предмету,
3) rm_tags(tags) для удаление тэга или нескольких тэгов предмета,
4) is_tagged() для проверки наличия одного или группы тэгов у предмета (если передана строка, то проверять один тэг, а если список, то несколько тэгов),
5) подсчет количества характеристик у предмета (переопределение "магического метода" __len__),
6) методы set_cost() и get_cost() для изменения и извлечения стоимости предмета.




# РЕАЛИЗАЦИЯ КЛАССОВ

*   Новый пункт
*   Новый пункт



In [17]:
import datetime

class Hub:
    _instance = None
    _items = []
    _id_counter = 1

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(Hub, cls).__new__(cls)
            cls._instance._date = datetime.date.today()
            cls._instance._items = []
        return cls._instance

    def add_item(self, name, description, date_add=None, tags=None, cost=0):
        if not name or not description:
            raise ValueError("Название и описание предмета обязательны")

        if cost < 0:
            raise ValueError("Стоимость не может быть отрицательной")

        if date_add is None:
            date_add = datetime.date.today()
        if tags is None:
            tags = []

        item_id = Hub._id_counter
        Hub._id_counter += 1
        item = Item(item_id, name, description, date_add, tags, cost)
        self._items.append(item)
        return item_id

    def remove_item(self, item_id):
        for i, item in enumerate(self._items):
            if item.get_id() == item_id:
                del self._items[i]
                return True
        return False

    def __getitem__(self, index):
        return self._items[index]

    def __len__(self):
        return len(self._items)

    def __str__(self):
        return f"Склад создан: {self._date}, Количество предметов: {len(self._items)}"

    def find_by_tags(self, tags):
        if isinstance(tags, str):
            tags = [tags]
        result = []
        for item in self._items:
            if item.is_tagged(tags):
                result.append(item)
        return result

    def set_date(self, date):
        self._date = date

    def get_date(self):
        return self._date

    def find_by_date(self, *args):
        if len(args) == 1:
            # Одна дата - все предметы с датой <= указанной
            target_date = args[0]
            return [item for item in self._items if item.get_date_add() <= target_date]
        elif len(args) == 2:
            # Две даты - предметы в промежутке
            start_date, end_date = args
            return [item for item in self._items if start_date <= item.get_date_add() <= end_date]
        else:
            raise ValueError("Метод принимает только 1 или 2 аргумента")

    def find_most_valuable(self, amount=1):
        sorted_items = sorted(self._items, key=lambda x: x.get_cost(), reverse=True)
        return sorted_items[:min(amount, len(sorted_items))]


class Item:
    def __init__(self, item_id, name, description, date_add, tags=None, cost=0):
        self._id = item_id
        self._name = name
        self._description = description
        self._date_add = date_add
        self._tags = tags if tags is not None else []
        self._cost = cost

    def get_id(self):
        return self._id

    def get_date_add(self):
        return self._date_add

    def get_cost(self):
        return self._cost

    def set_cost(self, cost):
        self._cost = cost

    def __str__(self):
        return f"ID: {self._id}, Название: {self._name}, Описание: {self._description}, Дата добавления: {self._date_add}, Теги: {self._tags}, Стоимость: {self._cost}"

    def add_tags(self, tags):
        if isinstance(tags, str):
            tags = [tags]
        for tag in tags:
            if tag not in self._tags:
                self._tags.append(tag)

    def rm_tags(self, tags):
        if isinstance(tags, str):
            tags = [tags]
        for tag in tags:
            if tag in self._tags:
                self._tags.remove(tag)

    def is_tagged(self, tags):
        if isinstance(tags, str):
            return tags in self._tags
        else:
            return all(tag in self._tags for tag in tags)

    def __len__(self):
        return len(self._tags)

# Создание объектов
1) Создайте склад и занести в него 10 люых предметов (предметы формировать прямо в коде программы, не вводя их с клавиатуры).
2) Выведите информацию про склад и про каждый предмет

In [18]:
hub = Hub()

hub.add_item("Стул", "Деревянный стул", datetime.date(2024, 1, 15), ["мебель", "дерево", "Россия"], 1500)
hub.add_item("Стол", "Офисный стол", datetime.date(2024, 1, 20), ["мебель", "дерево", "Китай"], 3000)
hub.add_item("Ноутбук", "Игровой ноутбук", datetime.date(2024, 2, 1), ["электроника", "хрупкий", "США"], 50000)
hub.add_item("Монитор", "27-дюймовый монитор", datetime.date(2024, 2, 5), ["электроника", "хрупкий", "Китай"], 15000)
hub.add_item("Книга", "Учебник по Python", datetime.date(2024, 2, 10), ["книга", "образование", "Россия"], 800)
hub.add_item("Телефон", "Смартфон", datetime.date(2024, 2, 15), ["электроника", "хрупкий", "Корея"], 25000)
hub.add_item("Рюкзак", "Туристический рюкзак", datetime.date(2024, 2, 20), ["аксессуар", "туризм", "Китай"], 2000)
hub.add_item("Наушники", "Беспроводные наушники", datetime.date(2024, 3, 1), ["электроника", "аудио", "Китай"], 3000)
hub.add_item("Клавиатура", "Механическая клавиатура", datetime.date(2024, 3, 5), ["электроника", "игры", "Китай"], 4000)
hub.add_item("Мышь", "Компьютерная мышь", datetime.date(2024, 3, 10), ["электроника", "игры", "Китай"], 1500)

print("Информация о складе:")
print(hub)
print()

print("Предметы на складе:")
for i, item in enumerate(hub._items):
    print(f"{i+1}. {item}")

Информация о складе:
Склад создан: 2025-10-22, Количество предметов: 10

Предметы на складе:
1. ID: 1, Название: Стул, Описание: Деревянный стул, Дата добавления: 2024-01-15, Теги: ['мебель', 'дерево', 'Россия'], Стоимость: 1500
2. ID: 2, Название: Стол, Описание: Офисный стол, Дата добавления: 2024-01-20, Теги: ['мебель', 'дерево', 'Китай'], Стоимость: 3000
3. ID: 3, Название: Ноутбук, Описание: Игровой ноутбук, Дата добавления: 2024-02-01, Теги: ['электроника', 'хрупкий', 'США'], Стоимость: 50000
4. ID: 4, Название: Монитор, Описание: 27-дюймовый монитор, Дата добавления: 2024-02-05, Теги: ['электроника', 'хрупкий', 'Китай'], Стоимость: 15000
5. ID: 5, Название: Книга, Описание: Учебник по Python, Дата добавления: 2024-02-10, Теги: ['книга', 'образование', 'Россия'], Стоимость: 800
6. ID: 6, Название: Телефон, Описание: Смартфон, Дата добавления: 2024-02-15, Теги: ['электроника', 'хрупкий', 'Корея'], Стоимость: 25000
7. ID: 7, Название: Рюкзак, Описание: Туристический рюкзак, Дата до

# Работа со складом
1. Вывести информацию про предмет с заданным индексом (ввести с клавиатуры)
2. Вывести общее количество элементов на складе
3. Удалить предмет с заданным id
4. Вывести все предметы, содержащие заданную группу тэгов
5. Вывести все предметы с датой поступления, лежащей в указанном диапазоне
6. Вывести три самых дорогих предмета
7. Установить новую дату создания склада и вывести информацию про нее  
8. Проверить, что склад существует в единственном числе (создать новый склад и сравнить их id).


In [19]:
print("=== РАБОТА СО СКЛАДОМ ===\n")

# 1. Вывести информацию про предмет с заданным индексом
while True:
    try:
        index = int(input(f"Введите индекс предмета (0-{len(hub)-1}): "))
        if 0 <= index < len(hub):
            print(f"Предмет с индексом {index}: {hub[index]}")
            break
        else:
            print(f"Неверный индекс! Введите число от 0 до {len(hub)-1}")
    except ValueError:
        print("Ошибка! Введите целое число")

# 2. Вывести общее количество элементов на складе
print(f"Общее количество предметов на складе: {len(hub)}")

# 3. Удалить предмет с заданным id
while True:
    try:
        item_id_to_remove = int(input("Введите ID предмета для удаления: "))
        if hub.remove_item(item_id_to_remove):
            print(f"Предмет с ID {item_id_to_remove} удален")
            break
        else:
            print(f"Предмет с ID {item_id_to_remove} не найден. Попробуйте еще раз.")
    except ValueError:
        print("Ошибка! Введите целое число для ID")

# 4. Вывести все предметы, содержащие заданную группу тэгов
tags_to_find = ["электроника", "хрупкий"]
print(f"Предметы с тегами {tags_to_find}:")
for item in hub.find_by_tags(tags_to_find):
    print(f"  - {item}")

# 5. Вывести все предметы с датой поступления в диапазоне
start_date = datetime.date(2024, 2, 1)
end_date = datetime.date(2024, 2, 28)
print(f"Предметы с датой поступления с {start_date} по {end_date}:")
for item in hub.find_by_date(start_date, end_date):
    print(f"  - {item}")

# 6. Вывести три самых дорогих предмета
print("Три самых дорогих предмета:")
for i, item in enumerate(hub.find_most_valuable(3)):
    print(f"  {i+1}. {item}")

# 7. Установить новую дату создания склада
new_date = datetime.date(2023, 12, 1)
hub.set_date(new_date)
print(f"Новая дата создания склада: {hub.get_date()}")

# 8. Проверить, что склад существует в единственном числе
hub2 = Hub()
print(f"ID первого склада: {id(hub)}")
print(f"ID второго склада: {id(hub2)}")
print(f"Это один и тот же объект: {hub is hub2}")

=== РАБОТА СО СКЛАДОМ ===

Введите индекс предмета (0-9): u
Ошибка! Введите целое число
Введите индекс предмета (0-9): %
Ошибка! Введите целое число
Введите индекс предмета (0-9): 6
Предмет с индексом 6: ID: 7, Название: Рюкзак, Описание: Туристический рюкзак, Дата добавления: 2024-02-20, Теги: ['аксессуар', 'туризм', 'Китай'], Стоимость: 2000
Общее количество предметов на складе: 10
Введите ID предмета для удаления: t
Ошибка! Введите целое число для ID
Введите ID предмета для удаления: 1
Предмет с ID 1 удален
Предметы с тегами ['электроника', 'хрупкий']:
  - ID: 3, Название: Ноутбук, Описание: Игровой ноутбук, Дата добавления: 2024-02-01, Теги: ['электроника', 'хрупкий', 'США'], Стоимость: 50000
  - ID: 4, Название: Монитор, Описание: 27-дюймовый монитор, Дата добавления: 2024-02-05, Теги: ['электроника', 'хрупкий', 'Китай'], Стоимость: 15000
  - ID: 6, Название: Телефон, Описание: Смартфон, Дата добавления: 2024-02-15, Теги: ['электроника', 'хрупкий', 'Корея'], Стоимость: 25000
Предм

# Работа с предметами
1. Для предмета с заданным индексом добавить произвольных два тэга и вывести про него информацию на экран
2. Для предмета с заданным индексом изменить стоимость и вывести новую стоимость на экран
3. Для предмета с заданным индексом удалить два указанных тэга и вывести информацию про предмет на экран
4. Для предмета с заданным индексом вывести количество его характеристик

In [21]:
print("\n=== РАБОТА С ПРЕДМЕТАМИ ===\n")

# Выбор предмета по индексу
while True:
    try:
        index = int(input(f"Введите индекс предмета для работы (0-{len(hub)-1}): "))
        if 0 <= index < len(hub):
            selected_item = hub[index]
            print(f"Выбран предмет: {selected_item._name}")
            break
        else:
            print(f"Неверный индекс! Введите число от 0 до {len(hub)-1}")
    except ValueError:
        print("Ошибка! Введите целое число")

print("\n=== РАБОТА С ПРЕДМЕТАМИ ===\n")

# 1. Добавить два тэга предмету
item = selected_item
print(f"Предмет до добавления тегов: {item}")
new_tags = input("Введите теги для добавления (через запятую): ").split(',')
new_tags = [tag.strip() for tag in new_tags if tag.strip()]
item.add_tags(new_tags)
print(f"Предмет после добавления тегов: {item}")

# 2. Изменить стоимость предмета
item = selected_item
while True:
    try:
        new_cost = float(input(f"Введите новую стоимость для '{item._name}': "))
        item.set_cost(new_cost)
        print(f"Новая стоимость предмета: {item.get_cost()}")
        break
    except ValueError:
        print("Ошибка! Введите число для стоимости")

# 3. Удалить тэги у предмета
item = selected_item
print(f"Текущие теги предмета: {item._tags}")
tags_to_remove = input("Введите теги для удаления (через запятую): ").split(',')
tags_to_remove = [tag.strip() for tag in tags_to_remove if tag.strip()]
item.rm_tags(tags_to_remove)
print(f"Предмет после удаления тегов: {item}")

# 4. Вывести количество характеристик предмета
item = selected_item
print(f"Количество характеристик у предмета '{item._name}': {len(item)}")


=== РАБОТА С ПРЕДМЕТАМИ ===

Введите индекс предмета для работы (0-8): ш
Ошибка! Введите целое число
Введите индекс предмета для работы (0-8): 5
Выбран предмет: Рюкзак

=== РАБОТА С ПРЕДМЕТАМИ ===

Предмет до добавления тегов: ID: 7, Название: Рюкзак, Описание: Туристический рюкзак, Дата добавления: 2024-02-20, Теги: ['аксессуар', 'туризм', 'Китай'], Стоимость: 2000
Введите теги для добавления (через запятую): 1,2.4
Предмет после добавления тегов: ID: 7, Название: Рюкзак, Описание: Туристический рюкзак, Дата добавления: 2024-02-20, Теги: ['аксессуар', 'туризм', 'Китай', '1', '2.4'], Стоимость: 2000
Введите новую стоимость для 'Рюкзак': 4400
Новая стоимость предмета: 4400.0
Текущие теги предмета: ['аксессуар', 'туризм', 'Китай', '1', '2.4']
Введите теги для удаления (через запятую): 1
Предмет после удаления тегов: ID: 7, Название: Рюкзак, Описание: Туристический рюкзак, Дата добавления: 2024-02-20, Теги: ['аксессуар', 'туризм', 'Китай', '2.4'], Стоимость: 4400.0
Количество характеристи